# Example 2: Computation of Gradients and Non-adiabatic Couplings

In this tutorial it is shown, how to use the SA-OO-VQE solver in a simple step-by-step manner on a molecule of
formaldimine (methylene imine). It is shown, how to compute energies, gradients and non-adiabatic couplings
in several different settings w.r.t. a different number of optimized orbitals.

First of all, we'll specify geometry of the system.

In [1]:
symbols = ['N', 'C', 'H', 'H', 'H']
coords = [
    [0.000000000000, 0.000000000000, 0.000000000000],
    [0.000000000000, 0.000000000000, 1.498047000000],
    [0.000000000000, -0.938765985000, 2.004775984000],
    [0.000000000000, 0.938765985000, 2.004775984000],
    [-0.744681452, -0.131307432, -0.634501434]
]

Now we'll specify its properties, active space and the basis for Psi4 chemistry backend.

In [2]:
n_orbs_active = 2
n_elec_active = 2
charge = 0
multiplicity = 1
basis = 'sto-3g'

The next step is construction of `ProblemSet` instance - object containing all the information and necessary method for
our electronic structure problem.

In [3]:
import saoovqe

problem = saoovqe.problem.ProblemSet(
    symbols=symbols,
    coords=coords,
    charge=charge,
    multiplicity=multiplicity,
    n_electrons_active=n_elec_active,
    n_orbitals_active=n_orbs_active,
    basis_name=basis
)

2024-09-18 01:18:42,039 - SAOOVQE.logger - INFO - SecondQuantizedProblem was created.


Now we need to create a set of circuits representing orthogonal states to construct the whole circuits representing
state vectors later.

In [4]:
initial_circuits = saoovqe.OrthogonalCircuitSet.from_problem_set(n_states=2, problem=problem)

2024-09-18 01:18:42,047 - SAOOVQE.logger - INFO - Circuits representing an orthogonal basis were created.


The next necessary part is to define an ansatz - it'll be also used to construct the state vector circuits later.

In [5]:
ansatz = saoovqe.Ansatz.from_problem_set(ansatz=saoovqe.AnsatzType.GUCCSD,
                                         problem=problem,
                                         repetitions=1,
                                         qubit_mapper=problem.fermionic_mapper)

2024-09-18 01:18:42,205 - SAOOVQE.logger - INFO - Ansatz was created.


And finally, now we can create an instance of our SA-OO-VQE solver. One of the main points is, orbital-optimization
can, but doesn't have to be used, or it can be used only on some molecular orbitals. We'll show all three cases here.
For no orbital-optimization it's enough to pass `None` to `orbital_optimization_settings` (it's also a default value).

In [6]:
from qiskit.primitives import Estimator, Sampler

estimator = Estimator()
sampler = Sampler()

solver_no_oo = saoovqe.SAOOVQE(estimator=estimator,
                               initial_circuits=initial_circuits,
                               ansatz=ansatz,
                               problem=problem,
                               orbital_optimization_settings=None)

2024-09-18 01:18:42,211 - SAOOVQE.logger - INFO - SAOOVQE was created.


To specify number of optimized orbitals we can pass a dictionary to the parameter.

In [7]:
solver_oo_8 = saoovqe.SAOOVQE(estimator=estimator,
                              initial_circuits=initial_circuits,
                              ansatz=ansatz,
                              problem=problem,
                              orbital_optimization_settings={'n_mo_optim': 8})

2024-09-18 01:18:42,217 - SAOOVQE.logger - INFO - SAOOVQE was created.


And for all the orbitals to be optimized we can simply use its default behavior by passing an empty dictionary.

In [8]:
solver_oo_full = saoovqe.SAOOVQE(estimator=estimator,
                                 initial_circuits=initial_circuits,
                                 ansatz=ansatz,
                                 problem=problem,
                                 orbital_optimization_settings={})

2024-09-18 01:18:42,221 - SAOOVQE.logger - INFO - SAOOVQE was created.


Let's compare the energies now! To make the numerical optimizations, we'll use `SLSQP` optimizer provided by `SciPy`.
It may take a few minutes time now...

In [9]:
from qiskit_algorithms.optimizers import SciPyOptimizer
import numpy as np

optimizer = SciPyOptimizer('SLSQP', options={'maxiter': 500, 'ftol': 1e-8})
energies_no_oo = solver_no_oo.get_energy(optimizer)
energies_oo_8 = solver_oo_8.get_energy(optimizer)
energies_oo_full = solver_oo_full.get_energy(optimizer)

print('\n============== State-Averaged Energies ==============')
print(np.mean(energies_no_oo))
print(np.mean(energies_oo_8))
print(np.mean(energies_oo_full))

2024-09-18 01:18:42,226 - SAOOVQE.logger - INFO - Computing energies...
2024-09-18 01:18:43,440 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.2726154   0.27261305 -0.01552257]
2024-09-18 01:18:43,876 - SAOOVQE.logger - INFO - Optimal phi angle for state-resolution was obtained (phi* = 3.806878207655641).
2024-09-18 01:18:44,359 - SAOOVQE.logger - INFO - Computing energies...
2024-09-18 01:18:45,631 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.2726154   0.27261305 -0.01552257]
2024-09-18 01:18:45,632 - SAOOVQE.logger - INFO - Starting Orbital-Optimization process...
2024-09-18 01:18:46,096 - SAOOVQE.logger - INFO - Starting orbital optimization (iteration 0)
2024-09-18 01:18:46,341 - SAOOVQE.logger - INFO - Gradient norm: 0.0356412262738926
2024-09-18 01:18:46,342 - SAOOVQE.logger - INFO - Energy after this OO iteration: -92.64374463947547
2024-09-18 01:18:46,343 - SAOOVQE.logger - INFO - Best energy achieved: -92.64374463947547
2024-09-18 01:18:46,343 -


============== State-Averaged Energies ==============
-92.64256593070509
-92.64414484063462
-92.65981061347676


And now, let's have a look at the gradients of the potential energy surface for all the particles at both relevant
states.

In [10]:
print('\n============== Gradients ==============')
for state_idx in range(2):
    for atom_idx in range(len(symbols)):
        print(state_idx, atom_idx, solver_oo_full.eval_eng_gradient(state_idx, atom_idx))

2024-09-18 01:20:45,833 - SAOOVQE.logger - INFO - Constructing gradient evaluators...



============== Gradients ==============
0 0 [-0.22492213 -0.0490333  -0.20123791]
0 1 [0.07455834 0.01793913 0.23635008]
0 2 [-0.0030157   0.02465403 -0.02605519]
0 3 [-0.00637749 -0.02588277 -0.02037206]
0 4 [0.15975698 0.03232291 0.01131508]
1 0 [-0.04098252 -0.02451307 -0.54292557]
1 1 [0.064886  0.0010373 0.5165189]
1 2 [-0.02694857  0.03139608 -0.02809479]
1 3 [-0.03869227 -0.0301688  -0.02735175]
1 4 [0.04173736 0.0222485  0.08185321]


And the non-adiabatic couplings.

In [11]:
print('\n============== Total non-adiabatic couplings ==============')
for atom_idx in range(len(symbols)):
    print(atom_idx, solver_oo_full.eval_nac(atom_idx))


============== Total non-adiabatic couplings ==============
0 [-0.5565272   2.35403065  1.28518241]
1 [-0.12877282 -1.09821038 -1.32250679]
2 [0.03282151 0.01785465 0.22158791]
3 [ 0.25561399  0.08504591 -0.10507403]
4 [ 0.36228558 -1.25132444 -0.07593426]


And finally, considering NACs, we can also have a look at CI and CSF NACs separately.

In [12]:
print('\n============== CI non-adiabatic couplings ==============')
for atom_idx in range(len(symbols)):
    print(atom_idx, solver_oo_full.ci_nacs[atom_idx])

print('\n============== CSF non-adiabatic couplings ==============')
for atom_idx in range(len(symbols)):
    print(atom_idx, solver_oo_full.csf_nacs[atom_idx])


============== CI non-adiabatic couplings ==============
0 [-0.51517591  2.1407781   1.28505707]
1 [-0.12202162 -1.06704725 -1.32160249]
2 [-0.01316415  0.0193005   0.2256887 ]
3 [ 0.29604036  0.08643814 -0.10915325]
4 [ 0.35432132 -1.17946949 -0.07999003]

============== CSF non-adiabatic couplings ==============
0 [-4.13512940e-02  2.13252546e-01  1.25340222e-04]
1 [-0.00675121 -0.03116313 -0.00090429]
2 [ 0.04598567 -0.00144585 -0.00410079]
3 [-0.04042637 -0.00139223  0.00407921]
4 [ 0.00796426 -0.07185495  0.00405577]
